## Import Statements

In [1]:
import os
import cv2
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from keras.utils import np_utils, generic_utils


from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses

from matplotlib import pyplot as plt
from random import randrange

In [1]:
!nvidia-smi

Tue Feb  8 17:25:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| 44%   54C    P2   103W / 250W |   9823MiB / 11264MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
#Number of subjects
print(os.listdir(subjectPath))

['s1', 's11', 's12', 's13', 's14', 's15', 's18', 's19', 's2', 's20', 's3', 's4', 's5', 's6', 's8', 's9']


In [2]:
image_rows, image_columns, frames_Count = 112, 112, 16
training_smtc_list_netA = []
training_smtc_list_netB = []

In [3]:
subjectPath = 'Datasets/SMIC/HS'
#subDirectory = os.listdir(subjectPath)

## Converting into arrays

In [19]:
for sub in subDirectory:
    framespath = 'Datasets/SMIC/HS/' + sub + '/micro/negative/'
    frames = os.listdir(framespath)
    #print(frames)
    
    for parts in frames:
        fm = os.listdir(framespath + parts)
        print(fm[1])
        break
    break

reg_image559741.bmp


# SMIC Datasset

### Model A

In [4]:
#Dataset has three different folders of micro-expressions
for lab in ['negative', 'positive', 'surprise']:
    subDirectory = os.listdir(subjectPath)
    count = 0    
    #From each fetching the subjects/persons
    for sub in subDirectory:
        labelpath = 'Datasets/SMIC/HS/'+ sub +'/micro/' + lab + '/'
        directorylisting = os.listdir(labelpath)
        #Each subject has multiple expression video files
        for video in directorylisting:
            videopath = labelpath + video
            frames = []
            framelisting = os.listdir(videopath)
            #Filtering the subjects having the frames than the frame count to avoid dimensionality problem 
            if len(framelisting) > frames_Count:
                framerange = [x for x in range(frames_Count)]
                count = count + 1
                #Pre-process every single frame
                for frame in framerange:
                    imagepath = videopath + "/" + framelisting[frame]
                    image = cv2.imread(imagepath)
                    imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                    # grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                    frames.append(imageresize)
                    # break
                frames = np.asarray(frames)
                videoarray = np.rollaxis(np.rollaxis(frames, 2, 0), 2, 0)
                training_smtc_list_netA.append(videoarray)
              # break
    #Video samples count for each expressions
    print(lab,count)
    # break

negative 69
positive 50
surprise 43


In [11]:
smtc_train_netA = np.asarray(training_smtc_list_netA)
smtc_train_netA = np.rollaxis(np.rollaxis(smtc_train_netA, 2, 4), 2, 1)
smtc_train_netA = (smtc_train_netA.astype('float32') / 255)

# Assign Labels
smtc_target_netA = np.zeros((len(smtc_train_netA), ), dtype = int)
smtc_target_netA[0:69] = 0
smtc_target_netA[69:119] = 1
smtc_target_netA[119:162] = 2

# Convert to one hot
# smtc_target = np_utils.to_categorical(smtc_target, 3)

In [12]:
#Number of sample, number of frames from each video, hieght, width, channels
smtc_train_netA.shape

(162, 16, 112, 112, 3)

In [13]:
# Save Nump Arrays to save time
np.save('training_datasets/microexp_2D_to_3D_imagesA.npy', smtc_train_netA)
np.save('training_datasets/microexp_2D_to_3D_labelsA.npy', smtc_target_netA)

### Model B

Model-B pre-processing is similar flow as model-A and obtaining the differential image(colarating all the images difference into single image)

In [15]:
training_smtc_list_netB = []
for lab in ['negative', 'positive', 'surprise']:
    subDirectory = os.listdir(subjectPath)
    count = 0
    for sub in subDirectory:
        labelpath = 'Datasets/SMIC/HS/'+ sub +'/micro/' + lab + '/'
        directorylisting = os.listdir(labelpath)
        for video in directorylisting:
            videopath = labelpath + video
            frames = []
            framelisting = os.listdir(videopath)
            if len(framelisting) > frames_Count:
                randomStartFrame = randrange(len(framelisting)-frames_Count)
                framerange = [x for x in range(randomStartFrame, (randomStartFrame+frames_Count))]
                count = count + 1
                for frame in framerange:
                    imagepath = videopath + "/" + framelisting[frame]
                    image = cv2.imread(imagepath)
                    imageresize = cv2.resize(image, (image_rows, image_columns), interpolation = cv2.INTER_AREA)
                    # grayimage = cv2.cvtColor(imageresize, cv2.COLOR_BGR2GRAY)
                    frames.append(imageresize)
                frames = np.asarray(frames)
                diff_info_Image = np.zeros(shape=[112,112,3])
                for indCount, images in enumerate(frames):
                    if indCount == 0:
                        #Do Nothing
                        continue
                    diff_info_Image = diff_info_Image + (np.absolute(images-frames[0]))
              # videoarray = np.rollaxis(np.rollaxis(frames, 2, 0), 2, 0)
                training_smtc_list_netB.append(diff_info_Image)
    print(lab,count)

negative 69
positive 50
surprise 43


In [16]:
smtc_train_netB = np.asarray(training_smtc_list_netB)
# smtc_train_netB = (smtc_train_netB.astype('float32') / 255)

# Assign Labels
smtc_target_netB = np.zeros((len(smtc_train_netB), ), dtype = int)
smtc_target_netB[0:69] = 0
smtc_target_netB[66:119] = 1
smtc_target_netB[113:162] = 2

# Convert to one hot
# smtc_target = np_utils.to_categorical(smtc_target, 3)

In [17]:
#Number of sample, hieght, width, channels
smtc_train_netB.shape

(162, 112, 112, 3)

#### Saving the Model

In [18]:
np.save('training_datasets/microexp_2D_to_3D_imagesB.npy', smtc_train_netB)
np.save('training_datasets/microexp_2D_to_3D_labelsB.npy', smtc_target_netB)

## Loading Model A

In [2]:
#Load the saved numpy data NetA
smtc_train_netA = None
smtc_target_netA = None
smtc_train_netA = np.load('training_datasets/microexp_2D_to_3D_imagesA.npy')
smtc_target_netA =np.load('training_datasets/microexp_2D_to_3D_labelsA.npy')

smtc_train_netA.shape

(162, 16, 112, 112, 3)

## Loading Model B

In [3]:
#Load the saved numpy data NetB
smtc_train_netB = None
smtc_target_netB = None
smtc_train_netB = np.load('training_datasets/microexp_2D_to_3D_imagesB.npy')
smtc_target_netB =np.load('training_datasets/microexp_2D_to_3D_labelsB.npy')

smtc_train_netB.shape

(162, 112, 112, 3)

## Train-test-split

In [4]:
x_train_A, x_test_A, x_train_B, x_test_B, y_train, y_test = train_test_split(smtc_train_netA, smtc_train_netB, smtc_target_netA, test_size=0.2, random_state=4)

y_train_A = y_train
y_test_A = y_test

y_train_B = y_train
y_test_B = y_test


# Convert Into Tensors
train_data_A = tf.data.Dataset.from_tensor_slices((x_train_A, y_train_A)).batch(8, drop_remainder=True)
test_data_A = tf.data.Dataset.from_tensor_slices((x_test_A, y_test_A)).batch(8, drop_remainder=True)

train_data_B = tf.data.Dataset.from_tensor_slices((x_train_B, y_train_B)).batch(8, drop_remainder=True)
test_data_B = tf.data.Dataset.from_tensor_slices((x_test_B, y_test_B)).batch(8, drop_remainder=True)

# Use the below dataset for model.fit
train_All_dataset = tf.data.Dataset.from_tensor_slices(({"NET_A_InputLayer": x_train_A, "NET_B_InputLayer": x_train_B}, y_train_A)).batch(8, drop_remainder=True)
test_All_dataset = tf.data.Dataset.from_tensor_slices(({"NET_A_InputLayer": x_test_A, "NET_B_InputLayer": x_test_B}, y_test_A)).batch(8, drop_remainder=True)


## Architecture of Model A

In [5]:
def _modelA(name="NET_A1"):
    input_shape = (16, 112, 112, 3)

  
    input = tf.keras.Input(shape=input_shape, name="NET_A_InputLayer")

      # Block 1
      # Convolution Layers
     
    conv_layer_1_1 = tf.keras.layers.Conv3D(filters=16,kernel_size=(1,3,3), strides=(1,2,2), padding="same", name="Conv3d_layer1_1")(input) 
    conv_layer_1_2 = tf.keras.layers.Conv3D(filters=16,kernel_size=(3,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer1_2")(input)
    conv_layer_1_3 = tf.keras.layers.Conv3D(filters=16,kernel_size=(5,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer1_3")(input)


      # Concatenation Layer
    concat_layer_1 = tf.keras.layers.concatenate([conv_layer_1_1, conv_layer_1_2, conv_layer_1_3], axis=1)


      # Block 2
    conv_layer_2_1 = tf.keras.layers.Conv3D(filters=16,kernel_size=(1,3,3), strides=(1,2,2), padding="same", name="Conv3d_layer2_1")(concat_layer_1)
    conv_layer_2_2 = tf.keras.layers.Conv3D(filters=16,kernel_size=(3,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer2_2")(concat_layer_1)
    conv_layer_2_3 = tf.keras.layers.Conv3D(filters=16,kernel_size=(5,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer2_3")(concat_layer_1)

      #Max Pool Layers
    conv_maxpool_layer_2_2 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer2_2")(conv_layer_2_2)
    conv_maxpool_layer_2_3 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer2_3")(conv_layer_2_3)

    concat_layer_2 = tf.keras.layers.concatenate([conv_layer_2_1, conv_maxpool_layer_2_2, conv_maxpool_layer_2_3], axis=1)

      # Block 3
    conv_layer_3_1 = tf.keras.layers.Conv3D(filters=32,kernel_size=(1,3,3), strides=(1,2,2), padding="same", name="Conv3d_layer3_1")(concat_layer_2)
    conv_layer_3_2 = tf.keras.layers.Conv3D(filters=32,kernel_size=(3,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer3_2")(concat_layer_2)
    conv_layer_3_3 = tf.keras.layers.Conv3D(filters=32,kernel_size=(5,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer3_3")(concat_layer_2)

    conv_maxpool_layer_3_2 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer3_2")(conv_layer_3_2)
    conv_maxpool_layer_3_3 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer3_3")(conv_layer_3_3)

    concat_layer_3 = tf.keras.layers.concatenate([conv_layer_3_1, conv_maxpool_layer_3_2, conv_maxpool_layer_3_3], axis=1)

      # Block 4
    conv_layer_4_1 = tf.keras.layers.Conv3D(filters=64,kernel_size=(1,3,3), strides=(1,2,2), padding="same", name="Conv3d_layer4_1")(concat_layer_3)
    conv_layer_4_2 = tf.keras.layers.Conv3D(filters=64,kernel_size=(3,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer4_2")(concat_layer_3)
    conv_layer_4_3 = tf.keras.layers.Conv3D(filters=64,kernel_size=(5,1,1), strides=(1,2,2), padding="same", name="Conv3d_layer4_3")(concat_layer_3)

    conv_maxpool_layer_4_2 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer4_2")(conv_layer_4_2)
    conv_maxpool_layer_4_3 = tf.keras.layers.MaxPool3D(pool_size=(2, 1, 1), name="Conv3d_maxpool_layer4_3")(conv_layer_4_3)

    concat_layer_4 = tf.keras.layers.concatenate([conv_layer_4_1, conv_maxpool_layer_4_2, conv_maxpool_layer_4_3], axis=1)

      # Block 5
    conv_layer_5 = tf.keras.layers.Conv3D(filters=128,kernel_size=(3,3,3), padding="same", name="Conv3d_layer5")(concat_layer_4)
    conv_maxpool_layer_5 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), padding="same", name="Conv3d_maxpool_layer5")(conv_layer_5)

    func_model = tf.keras.Model(input, conv_maxpool_layer_5, name=name)

    return func_model

In [6]:
model_A = _modelA()

In [7]:
# model_A.summary()
tf.keras.utils.plot_model(model_A, "NETA.png", show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [8]:
#pip install pydot

## Architecture of Model B

In [9]:
def _modelB(name="NET_B1"):
    input_shape = (112, 112, 3)


    input = tf.keras.Input(shape=input_shape, name="NET_B_InputLayer")

    x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), padding="same", name="Res_Layer1")(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x_skip = x

      # Residual Block 1
    x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), padding="same", name="Res_Block_Layer1_1")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), padding="same", name="Res_Block_Layer1_2")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name="Layer1_Avg_Pool")(x)
    x_skip = x


      # Residual Block 2
    x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), padding="same", name="Res_Block_Layer2_1")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), padding="same", name="Res_Block_Layer2_2")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name="Layer2_Avg_Pool")(x)

    x_skip = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3), padding="same", name="Res_Layer2")(x)

      # Residual Block 3
    x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3), padding="same", name="Res_Block_Layer3_1")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3), padding="same", name="Res_Block_Layer3_2")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name="Layer3_Avg_Pool")(x)


    x_skip = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3), padding="same", name="Res_Layer3")(x)

      # Residual Block 4
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3), padding="same", name="Res_Block_Layer4_1")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3), padding="same", name="Res_Block_Layer4_2")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name="Layer4_Avg_Pool")(x)

    x_skip = tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3), padding="same", name="Res_Layer4")(x)

      # Residual Block 5
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3), padding="same", name="Res_Block_Layer5_1")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3), padding="same", name="Res_Block_Layer5_2")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), padding="same", name="Layer5_Avg_Pool")(x)


    func_model = tf.keras.Model(input, x, name=name)

    return func_model

In [10]:
model_B = _modelB()

In [11]:
# model_A.summary()
tf.keras.utils.plot_model(model_B, "NETB.png", show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Concat the Architecture of Model A & Model B

In [12]:
def _modelAB(name="NET_AB"):
    #NetA
    NetAInput = model_A.input
    NetA_Output = model_A.layers[-1].output

      #NetB
    NetBInput = model_B.input
    NetB_Output = model_B.layers[-1].output

      #Add
    added = tf.keras.layers.Add(name="Add")([NetA_Output, NetB_Output])

    flatten = tf.keras.layers.Flatten(name="Flatten")(added)

    full_Connected = tf.keras.layers.Dense(units=3, name="Fully_Connected")(flatten)

    func_model = tf.keras.Model([NetAInput, NetBInput], full_Connected, name=name)

    return func_model

In [13]:
model_AB = _modelAB()

In [14]:
tf.keras.utils.plot_model(model_AB, "NETAB.png", show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Model Training

In [15]:
optimizer = tf.optimizers.Adam(learning_rate=1e-3)  # tune this
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
EPOCHS = 30

for epoch in range(EPOCHS):
    print("Epoch : ", epoch+1)
    for step, ((image_batch1, label_batch1),(image_batch2, label_batch2)) in enumerate(zip(train_data_A,train_data_B)):
        with tf.GradientTape() as tape:
            logits = model_AB([image_batch1, image_batch2])
            # loss format is generally: first argument targets, second argument outputs
            loss = loss_fn(label_batch1, logits)

        variables = model_AB.trainable_variables
        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))
        
        train_acc_metric(label_batch1, logits)
        if not step % 4:
            print("Step: {} Loss: {} Accuracy: {}".format(step, loss, train_acc_metric.result()))
            train_acc_metric.reset_states()

Epoch :  1
Step: 0 Loss: 207.6307830810547 Accuracy: 0.25
Step: 4 Loss: 772.58544921875 Accuracy: 0.21875
Step: 8 Loss: 122.43148803710938 Accuracy: 0.375
Step: 12 Loss: 34.019657135009766 Accuracy: 0.4375
Epoch :  2
Step: 0 Loss: 40.51036071777344 Accuracy: 0.4375
Step: 4 Loss: 26.190982818603516 Accuracy: 0.15625
Step: 8 Loss: 7.31225061416626 Accuracy: 0.28125
Step: 12 Loss: 2.9041051864624023 Accuracy: 0.3125
Epoch :  3
Step: 0 Loss: 1.249194622039795 Accuracy: 0.4375
Step: 4 Loss: 1.8004484176635742 Accuracy: 0.375
Step: 8 Loss: 1.339045763015747 Accuracy: 0.40625
Step: 12 Loss: 1.629081130027771 Accuracy: 0.3125
Epoch :  4
Step: 0 Loss: 0.9721174240112305 Accuracy: 0.46875
Step: 4 Loss: 0.7025880813598633 Accuracy: 0.375
Step: 8 Loss: 0.897973895072937 Accuracy: 0.53125
Step: 12 Loss: 1.3413565158843994 Accuracy: 0.3125
Epoch :  5
Step: 0 Loss: 1.1992677450180054 Accuracy: 0.46875
Step: 4 Loss: 0.5311616659164429 Accuracy: 0.53125
Step: 8 Loss: 0.8291283249855042 Accuracy: 0.5312

## Evaluating the Model

In [16]:
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for  step, ((image_batch1, label_batch1),(image_batch2, label_batch2)) in enumerate(zip(test_data_A,test_data_B)):
    test_acc_metric(label_batch1, model_AB([image_batch1, image_batch2]))
print("Test acc: {}".format(test_acc_metric.result()))

Test acc: 0.625
